# Assignment 7.  
## Aggregation queries

In this assigment, we will fetch data from a relational database server directly into Python variables. Using these new skills, we will complete the problems from Chapters 12, 13, and 14 in *SQL Queries for Mere Mortals*.

# Concepts

Review the MySQL aggregate functions: https://dev.mysql.com/doc/refman/8.0/en/group-by-functions.html

Three types of queries

1. Aggregation functions with no `GROUP BY` clause produce 1 row. 
2. Aggregation functions combined with a `GROUP BY` clause. The unique key of the result is composed of the columns of the `GROUP BY` clause.
3. `INNER JOIN` or `LEFT JOIN` of a tables in a one-to-many relationship, grouped by the primary key of the left table. This aggregates the right entity set with respect to the left entity set. 

Note that MySQL with the default settings allows mixing aggregated and non-aggregated values (See https://dev.mysql.com/doc/refman/5.7/en/sql-mode.html#sqlmode_only_full_group_by). So you have to watch avoid invalid mixes of values.

Using `HAVING` is equivalent to using a `WHERE` clause in an outer query.

# Aggregation Queries

Queries using aggregation functions, `GROUP BY`, and `HAVING` clauses. Using `LEFT JOIN` in combination with `GROUP BY`

In [ ]:
import json
import pymysql 

pymysql.install_as_MySQLdb()

with open('cred.json') as f:
    creds = json.load(f)

connection_string = "mysql://{user}:{password}@{host}".format(**creds)

In [ ]:
%load_ext sql
%config SqlMagic.autocommit=True
%sql $connection_string

### Increasing complexity of queries

In [ ]:
%%sql

USE shared_sales

## Aggregation functions MAX, MIN, AVG, SUM, COUNT

In [ ]:
%%sql
SELECT MAX(RetailPrice) as max_price, round(AVG(RetailPrice), 4) as avg_price 
FROM Products

In [ ]:
%%sql

SELECT ProductNumber, MAX(RetailPrice) as max_price, AVG(RetailPrice) as avg_price 
FROM Products

In [ ]:
%%sql

-- aggregation with condition

SELECT avg(RetailPrice) avg_price 
FROM Products 
WHERE CategoryID IN (
    SELECT CategoryID 
    FROM Categories
    WHERE CategoryDescription = "Bikes")

In [ ]:
%%sql
-- show products above average price
-- The following does NOT work

SELECT * 
FROM Products 
WHERE RetailPrice > avg(RetailPrice)

In [ ]:
%%sql
-- show products above average price
-- Use a scalar subquery

SELECT * 
FROM Products 
WHERE RetailPrice > (SELECT avg(RetailPrice) FROM Products)

In [ ]:
%%sql

-- what not to do:

SELECT *
FROM Products
GROUP BY CategoryID


In [ ]:
%%sql

SELECT CategoryID, ROUND(AVG(RetailPrice), 2) as avg_price
FROM Products
GROUP BY CategoryID


In [ ]:
%%sql 

-- using in a subquery

SELECT * FROM (
    SELECT CategoryID, AVG(RetailPrice) as avg_price
    FROM Products
    GROUP BY CategoryID) as q NATURAL JOIN Categories


In [ ]:
%%sql 

-- using in a subquery

SELECT Categories.*, AVG(RetailPrice) as avg_price 
FROM Categories NATURAL JOIN Products
GROUP BY CategoryID

In [ ]:
%%sql

-- show averages prices per category categories for products with retail price > 100.00

SELECT Categories.*, AVG(RetailPrice) as avg_price 
FROM Categories NATURAL JOIN Products 
WHERE RetailPrice > 100
GROUP BY CategoryID



In [ ]:
%%sql

-- show averages prices per category categories for products with retail price > 100.00

SELECT Categories.*, AVG(RetailPrice) as avg_price 
FROM Categories NATURAL JOIN Products
GROUP BY CategoryID
HAVING avg_price > 100


In [ ]:
%%sql

-- Categories with the average prices > 100

SELECT * FROM (
    SELECT Categories.*, AVG(RetailPrice) as avg_price 
    FROM Categories NATURAL JOIN Products
    GROUP BY CategoryID) as q
WHERE avg_price > 100

In [ ]:
%%sql

SELECT * FROM (
    SELECT CategoryID, AVG(RetailPrice) as avg_price
    FROM Products
    GROUP BY CategoryID
) as q WHERE avg_price > 100

In [ ]:
%%sql

select * from Products
WHERE RetailPrice > 100

In [ ]:
%%sql
-- show empty orders 

SELECT * 
FROM Orders 
WHERE OrderNumber NOT IN (
    SELECT OrderNumber 
    FROM Order_Details)

In [ ]:
%%sql
-- show empty orders

SELECT Orders.* 
FROM Orders NATURAL LEFT JOIN Order_Details 
WHERE ProductNumber IS NULL

In [ ]:
%%sql

-- bad because mixes aggregated and random 

SELECT * FROM Orders  NATURAL LEFT JOIN Order_Details
GROUP BY OrderNumber

In [ ]:
%%sql

-- show empty orders

SELECT Orders.*, COUNT(ProductNumber) as nitems 
FROM Orders NATURAL LEFT JOIN Order_Details
GROUP BY OrderNumber
HAVING nitems=0

In [ ]:
%%sql

SELECT orders.*, ProductNumber as nitems FROM orders  NATURAL LEFT JOIN order_details

## Example problems

In [ ]:
%%sql

-- Customers and the number of orders that they made

SELECT CustomerID, CustFirstName, CustLastName, count(OrderNumber) as norders
FROM Customers NATURAL LEFT JOIN Orders
GROUP BY CustomerID
ORDER BY norders

In [ ]:
%%sql

-- orders with their total amounts

SELECT OrderNumber, OrderDate, IFNULL(sum(QuotedPrice * QuantityOrdered), 0) as total_amount
FROM Orders NATURAL LEFT JOIN Order_Details 
GROUP BY OrderNumber
ORDER BY total_amount

In [ ]:
%%sql

-- customers and the total $$$ spent

SELECT CustomerID, CustLastName, sum(total_amount) money_spent
FROM customers NATURAL LEFT JOIN (
    SELECT CustomerID, IFNULL(sum(QuotedPrice * QuantityOrdered), 0) as total_amount
    FROM Orders NATURAL LEFT JOIN Order_Details 
    GROUP BY OrderNumber) as q
GROUP BY CustomerID
ORDER BY money_spent DESC


In [ ]:
%%sql

-- customers and the total $$$ spent

SELECT CustomerID, CustLastName, IFNULL(sum(QuotedPrice * QuantityOrdered), 0) money_spent
FROM Customers NATURAL LEFT JOIN (SELECT * FROM Orders NATURAL JOIN Order_Details) as q
GROUP BY CustomerID
ORDER BY money_spent DESC

# Summary of principles 
1. Without a `GROUP BY`, aggregation functions collapse the table into a single row.
2. With `GROUP BY`, the grouping attributes become the new primary key of the result.  
3. Do not mix aggregated and non-aggregated values in the result with or without a `GROUP BY`.
4. `HAVING` plays the same role as the `WHERE` clause in a nesting outer query so it can use the output of the aggregation functions.
5. `LEFT JOIN` is often follwed with a `GROUP BY` by the primary key attributes of the left table. In this scenario the entities in the right table are aggregated for each matching row in the left table.


### Chapter 12 (total aggregation -- no grouping)


In [ ]:
Problem 1. 
“How many customers do we have in the state of California?”

Viescas, John L.. SQL Queries for Mere Mortals (p. 460). Pearson Education. Kindle Edition. 

In [ ]:
“What is the average retail price of a mountain bike?” 

Viescas, John L.. SQL Queries for Mere Mortals (p. 467). Pearson Education. Kindle Edition. 

In [ ]:
“What was the date of our most recent order?”

Viescas, John L.. SQL Queries for Mere Mortals (p. 467). Pearson Education. Kindle Edition. 

In [ ]:
%%sql 
SELECT OrderNumber, OrderDate as latest_Order
FROM Orders
Order By latest_order DESC
LIMIT 1

In [ ]:
%%sql
SELECT max(OrderDate) as last_date 
FROM Orders

In [ ]:
%%sql
-- Where was the Last order to a customer in California

SELECT max(OrderDate) as last_date
FROM Orders
WHERE CustomerID in (SELECT CustomerID FROM Customers WHERE CustState="TX")

### Chapter 13 (aggregation with grouping)

In [ ]:
“Display for each product the product name and the total sales.”

Viescas, John L.. SQL Queries for Mere Mortals (p. 490). Pearson Education. Kindle Edition. 

### Chapter 14 (aggregation with grouping and filtering with HAVING)

In [ ]:
“Show me the subject categories that have fewer than three full professors teaching that subject.”

Viescas, John L.. SQL Queries for Mere Mortals (p. 513). Pearson Education. Kindle Edition. 

In [ ]:
%%sql
use shared_school


In [ ]:
%%sql

SELECT Categories.*, count(StaffID) as n 
   FROM Categories NATURAL LEFT JOIN (
       SELECT * FROM Faculty_Categories NATURAL JOIN Faculty WHERE Title="Professor") as q
   GROUP BY CategoryID
   HAVING n < 3

In [ ]:
“How many orders are for only one product?”

Viescas, John L.. SQL Queries for Mere Mortals (p. 528). Pearson Education. Kindle Edition. 

In [ ]:
%%sql
use shared_sales

In [ ]:
%%sql

SELECT count(*) FROM (
    SELECT Orders.*, count(ProductNumber) as n
       FROM Orders NATURAL LEFT JOIN Order_Details
        GROUP BY OrderNumber
        HAVING n=1) as q


In [ ]:
%%sql


# Homework --- Due Nov 12

## Chapter 12

**Problem 1 (Sales):** Show the total amount of order number 8.
    
    Answer $1492.60

**Problem 2 (School):** List the most senior staff members (earliest hired).
    
    Answer 1 row.

## Chapter 13

**Problem 3 (Sales):** List vendors along with the average number of days they take to deliver the order.
    
    Answer 10 rows.

**Problem 4 (Sales):** List vendors along with the number of products each sells.
    
    Answer 10 rows.

**Problem 5 (School):** List staff members and the number of classes each is scheduled to teach.
    
    Answer 27 rows.

## Chapter 14

**Problem 6 (Sales):** Count the orders that only have one product
    
    1 row

**Problem 7 (School):**  List staff members that teach fewer than three classes
    
    7 rows

**Problem 8 (Recipes):** List recipes that use more than three teaspoons of salt.
    
    1 row